In [1]:
from fastai.vision.all import *
from fastai.learner import *
from fastai.data.all import *
from fastai.callback.tracker import SaveModelCallback
import pandas as pd
import matplotlib.pyplot as plt
from pathlib2 import Path
import numpy as np
import random
from torch.nn import MSELoss

In [2]:
%%time

df = pd.read_csv('data/examples.csv')
df.shape

CPU times: user 25.6 s, sys: 3.53 s, total: 29.2 s
Wall time: 30.9 s


(17937758, 9)

In [3]:
%%time
fn2features = pd.read_pickle('data/fn2feature.pkl')

CPU times: user 18.3 s, sys: 16.1 s, total: 34.4 s
Wall time: 2min 20s


In [4]:
df.head()

,source_word,target_word,source_fn,target_fn,set_name,speaker_id,book_id,distance_from_target,audio_fpath
0,I,FELT,8af4aebcf4a74004b02db9f88d99e89a,1cb9442ec1a6468282da309756e2ff57,train-clean-360,7000,83696,1,data/LibriSpeech/train-clean-360/7000/83696/7000-83696-0000.flac
1,I,THAT,8af4aebcf4a74004b02db9f88d99e89a,2f60546c930c47068ee0a129e6d51c39,train-clean-360,7000,83696,2,data/LibriSpeech/train-clean-360/7000/83696/7000-83696-0000.flac
2,FELT,I,1cb9442ec1a6468282da309756e2ff57,8af4aebcf4a74004b02db9f88d99e89a,train-clean-360,7000,83696,1,data/LibriSpeech/train-clean-360/7000/83696/7000-83696-0000.flac
3,FELT,THAT,1cb9442ec1a6468282da309756e2ff57,2f60546c930c47068ee0a129e6d51c39,train-clean-360,7000,83696,1,data/LibriSpeech/train-clean-360/7000/83696/7000-83696-0000.flac
4,FELT,IT,1cb9442ec1a6468282da309756e2ff57,155ad336d88c4cbf814a1237983b5b18,train-clean-360,7000,83696,2,data/LibriSpeech/train-clean-360/7000/83696/7000-83696-0000.flac


In [9]:
%%time
fn2feature_length = {}

for fn, feature in fn2features.items():
    fn2feature_length[fn] = feature.shape[0]

CPU times: user 2.66 s, sys: 120 ms, total: 2.78 s
Wall time: 2.78 s


In [15]:
%%time

df['source_length'] = df.source_fn.apply(lambda fn: fn2feature_length[fn])
df['target_length'] = df.target_fn.apply(lambda fn: fn2feature_length[fn])

CPU times: user 17.4 s, sys: 692 ms, total: 18.1 s
Wall time: 18 s


In [30]:
df.to_csv('data/examples_with_length.csv', index=False)

In [21]:
((df.source_length < 70) & (df.target_length < 70)).mean()

0.9451823354958853

In [31]:
df.source_word.nunique()

66663

In [32]:
df[((df.source_length < 70) & (df.target_length < 70))].source_word.nunique()

56746

In [64]:
df[((df.source_length < 70) & (df.target_length < 70) & (df.source_length > 25))].drop_duplicates(subset='source_fn').sort_values('source_length', ascending=True).head(20)

,source_word,target_word,source_fn,target_fn,set_name,speaker_id,book_id,distance_from_target,audio_fpath,source_length,target_length
11974301,AROUND,A,74fb4099182b482396dbc9ab97e171e7,d0486e2a68d94072920b23614c98b0ab,train-clean-360,836,130900,2,data/LibriSpeech/train-clean-360/836/130900/836-130900-0057.flac,26,9
5245095,FAR,CAME,aa73322c9e10462a978722a3f72b0044,decaec35b74245b18929ccbefc7b90f9,train-clean-360,6519,69412,2,data/LibriSpeech/train-clean-360/6519/69412/6519-69412-0023.flac,26,29
1174315,GLARE,IN,04c5f0d7a6ef472da1164a4290cf3475,d8dec547cf2a4974a65b8c46aff46c1e,train-clean-360,3307,145138,2,data/LibriSpeech/train-clean-360/3307/145138/3307-145138-0060.flac,26,12
572955,DOWN,FELL,0782c2652a654f82801e5c0e0a7259d8,a5347293aafd46299f1d0a419976f9fa,train-clean-360,8684,293663,2,data/LibriSpeech/train-clean-360/8684/293663/8684-293663-0014.flac,26,24
5245083,SHE,THIS,f63f337279fc434294df60d3f370104a,3601078e79ee4aa18f44e98e58590d5c,train-clean-360,6519,69412,2,data/LibriSpeech/train-clean-360/6519/69412/6519-69412-0023.flac,26,33
17096265,GOT,WHAT,b05d0d93a19b4e3c8726a96b63c33a49,70df9a1d82384e9bbd13901259b3d03a,train-clean-100,7264,92310,2,data/LibriSpeech/train-clean-100/7264/92310/7264-92310-0020.flac,26,20
9835263,DAY,IN,c41140e1dbe945b8acbc4617919adfad,2552d490aeab44f1a51e0f826c5c7f14,train-clean-360,2816,4136,2,data/LibriSpeech/train-clean-360/2816/4136/2816-4136-0048.flac,26,7
2409649,PRETTY,HAS,84984a74befd4cf09e58c78600754859,f092f61e34374f05861d7228637e6124,train-clean-360,1112,128138,2,data/LibriSpeech/train-clean-360/1112/128138/1112-128138-0005.flac,26,20
13222745,SAW,CIRCLED,6f06b9b0e3f0416cb87916bc850df7d5,b28a08a9a0a94cf2947e1a86d2fd5416,train-clean-360,2194,141248,2,data/LibriSpeech/train-clean-360/2194/141248/2194-141248-0017.flac,26,59
11103984,HIM,WRITE,8673ec44a32140a09272133796bf7126,10b4b3716d364fa7b7cc5a9b2823a9e0,train-clean-360,2010,121416,2,data/LibriSpeech/train-clean-360/2010/121416/2010-121416-0021.flac,26,29


In [35]:
from IPython.lib.display import Audio

In [70]:
Audio(filename='data/examples_audio/8673ec44a32140a09272133796bf7126.wav')

In [74]:
df[((df.source_length < 70) & (df.target_length < 70) & (df.source_length > 20))].source_word.nunique()

56518